In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Aug 26 09:58:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D

In [ ]:
import cv2

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Training data/train.csv')

In [ ]:
train.head()

,Unnamed: 0,filename,emotion,durations
0,0,18777.mp3,neutral,2.44000
1,1,24041.mp3,neutral,3.36000
2,2,1621.mp3,joy,0.34068
3,3,28883.mp3,neutral,1.56000
4,4,14918.mp3,neutral,2.82000


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Training data/test.csv')

In [ ]:
X = np.load('/content/drive/MyDrive/Training data/trainimages.npy')

In [ ]:
emotion_dict = {}
i = 0
for emotion in train['emotion'].unique():
    emotion_dict[emotion] = i
    i += 1

Y = train['emotion'].apply(lambda x: emotion_dict[x]).to_numpy()

In [ ]:
X.shape

(5816, 150, 150, 3)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

In [ ]:
base_model_1 = ResNet50(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')
len(base_model_1.layers)

94781440/94765736 [==============================] - 1s 0us/step


175

In [ ]:
l1 = tf.keras.layers.Flatten()(base_model_1.output)
l1 = tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.L2(0.01), activation='relu')(l1)
l1 = tf.keras.layers.BatchNormalization(
    axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,
    beta_initializer='zeros', gamma_initializer='ones',
    moving_mean_initializer='zeros',
    moving_variance_initializer='ones', beta_regularizer=None,
    gamma_regularizer=None, beta_constraint=None, gamma_constraint=None,
    renorm=False, renorm_clipping=None, renorm_momentum=0.99, fused=None,
    trainable=True, virtual_batch_size=None, adjustment=None, name=None)(l1)
#l1 = tf.keras.layers.Dropout(0.2)(l1)
l1 = tf.keras.layers.Dense(7, activation='sigmoid')(l1)

model_1 = tf.keras.models.Model(base_model_1.input, l1)
model_1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)
optimizer = Adam(learning_rate=lr_schedule,beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model_1.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model_1.fit(X_train, Y_train, epochs=4, batch_size=64, validation_data=(X_test, Y_test))

In [ ]:
model = InceptionResNetV2(weights='imagenet', include_top=False)

219070464/219055592 [==============================] - 3s 0us/step


In [ ]:
model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization_1[0][0]      
________________________________________________________________________________

In [ ]:
sub = np.load('/content/drive/MyDrive/Training data/testdata.npy')

In [ ]:
sub = sub[..., np.newaxis]

In [ ]:
pred = model.predict_classes(sub)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
pred

array([0, 0, 0, ..., 6, 2, 0])

In [ ]:
test

,filename
0,26199.mp3
1,692.mp3
2,16821.mp3
3,1608.mp3
4,47947.wav
...,...
2487,16692.mp3
2488,29505.mp3
2489,5807.mp3
2490,43982.wav


In [ ]:
submission_file = pd.DataFrame()
submission_file['filename'] = test['filename']
inv_emotion_dict = {v: k for k, v in emotion_dict.items()}

pred_emotion = np.array([inv_emotion_dict[i] for i in pred])

In [ ]:
submission_file['emotion'] = pred_emotion

In [ ]:
submission_file

,filename,emotion
0,26199.mp3,neutral
1,692.mp3,neutral
2,16821.mp3,neutral
3,1608.mp3,neutral
4,47947.wav,disgust
...,...,...
2487,16692.mp3,neutral
2488,29505.mp3,neutral
2489,5807.mp3,anger
2490,43982.wav,disgust


In [ ]:
submission_file.to_csv('submission.csv', index=False)